# FizzBuzz

FizzBuzz是一个简单的小游戏。游戏规则如下：从1开始往上数数，当遇到3的倍数的时候，说fizz，当遇到5的倍数，说buzz，当遇到15的倍数，就说fizzbuzz，其他情况下则正常数数。

我们可以写一个简单的小程序来决定要返回正常数值还是fizz, buzz 或者 fizzbuzz。

In [1]:
# One-hot encode the desired outputs: [number, "fizz", "buzz", "fizzbuzz"]
def fizz_buzz_encode(i):
    if   i % 15 == 0: return 3
    elif i % 5  == 0: return 2
    elif i % 3  == 0: return 1
    else:             return 0
    
def fizz_buzz_decode(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction] # 第一个[]为list，第二个[]为列表索引，相当于list[i]

print(fizz_buzz_decode(1, fizz_buzz_encode(1)))
print(fizz_buzz_decode(2, fizz_buzz_encode(2)))
print(fizz_buzz_decode(5, fizz_buzz_encode(5)))
print(fizz_buzz_decode(12, fizz_buzz_encode(12)))
print(fizz_buzz_decode(15, fizz_buzz_encode(15)))

1
2
buzz
fizz
fizzbuzz


我们首先定义模型的输入与输出(训练数据)

In [2]:
import torch
import numpy as np

In [3]:
NUM_DIGITS = 10

# Represent each input by an array of its binary digits.
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])[::-1]

trX = torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])
trY = torch.LongTensor([fizz_buzz_encode(i) for i in range(101, 2 ** NUM_DIGITS)])

然后我们用PyTorch定义模型

In [4]:
# Define the model
NUM_HIDDEN = 100
model = torch.nn.Sequential(
    torch.nn.Linear(NUM_DIGITS, NUM_HIDDEN),
    torch.nn.ReLU(),
    torch.nn.Linear(NUM_HIDDEN, 4)
)

In [5]:
if torch.cuda.is_available():
    model = model.cuda()

- 为了让我们的模型学会FizzBuzz这个游戏，我们需要定义一个损失函数，和一个优化算法。
- 这个优化算法会不断优化（降低）损失函数，使得模型的在该任务上取得尽可能低的损失值。
- 损失值低往往表示我们的模型表现好，损失值高表示我们的模型表现差。
- 由于FizzBuzz游戏本质上是一个分类问题，我们选用Cross Entropyy Loss函数。
- 优化函数我们选用Stochastic Gradient Descent。

In [6]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.05)

以下是模型的训练代码

In [7]:
# Start training it
BATCH_SIZE = 128
for epoch in range(10000):
    if torch.cuda.is_available():
        trX = trX.cuda()
        trY = trY.cuda() 
    for start in range(0, len(trX), BATCH_SIZE):
        end = start + BATCH_SIZE
        batchX = trX[start:end]
        batchY = trY[start:end]

        y_pred = model(batchX)
        loss = loss_fn(y_pred, batchY)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Find loss on training data
    loss = loss_fn(model(trX), trY).item()
    print('Epoch:', epoch, 'Loss:', loss)

Epoch: 0 Loss: 1.1756057739257812
Epoch: 1 Loss: 1.1549040079116821
Epoch: 2 Loss: 1.1490532159805298
Epoch: 3 Loss: 1.1465835571289062
Epoch: 4 Loss: 1.1452746391296387
Epoch: 5 Loss: 1.1444536447525024
Epoch: 6 Loss: 1.143872618675232
Epoch: 7 Loss: 1.1434226036071777
Epoch: 8 Loss: 1.143049716949463
Epoch: 9 Loss: 1.1427260637283325
Epoch: 10 Loss: 1.142439365386963
Epoch: 11 Loss: 1.142177939414978
Epoch: 12 Loss: 1.14193594455719
Epoch: 13 Loss: 1.1417114734649658
Epoch: 14 Loss: 1.1414992809295654
Epoch: 15 Loss: 1.141297698020935
Epoch: 16 Loss: 1.141106367111206
Epoch: 17 Loss: 1.1409220695495605
Epoch: 18 Loss: 1.1407465934753418
Epoch: 19 Loss: 1.1405762434005737
Epoch: 20 Loss: 1.1404134035110474
Epoch: 21 Loss: 1.1402548551559448
Epoch: 22 Loss: 1.1401033401489258
Epoch: 23 Loss: 1.139954924583435
Epoch: 24 Loss: 1.1398123502731323
Epoch: 25 Loss: 1.1396729946136475
Epoch: 26 Loss: 1.1395386457443237
Epoch: 27 Loss: 1.1394096612930298
Epoch: 28 Loss: 1.1392844915390015
Epoc

Epoch: 245 Loss: 1.1200323104858398
Epoch: 246 Loss: 1.1198550462722778
Epoch: 247 Loss: 1.1196728944778442
Epoch: 248 Loss: 1.1194865703582764
Epoch: 249 Loss: 1.1193028688430786
Epoch: 250 Loss: 1.1191128492355347
Epoch: 251 Loss: 1.1189247369766235
Epoch: 252 Loss: 1.1187379360198975
Epoch: 253 Loss: 1.1185451745986938
Epoch: 254 Loss: 1.1183476448059082
Epoch: 255 Loss: 1.1181576251983643
Epoch: 256 Loss: 1.1179654598236084
Epoch: 257 Loss: 1.11776864528656
Epoch: 258 Loss: 1.1175718307495117
Epoch: 259 Loss: 1.1173689365386963
Epoch: 260 Loss: 1.117173671722412
Epoch: 261 Loss: 1.1169712543487549
Epoch: 262 Loss: 1.1167744398117065
Epoch: 263 Loss: 1.1165742874145508
Epoch: 264 Loss: 1.116381287574768
Epoch: 265 Loss: 1.1161792278289795
Epoch: 266 Loss: 1.115975022315979
Epoch: 267 Loss: 1.1157722473144531
Epoch: 268 Loss: 1.1155569553375244
Epoch: 269 Loss: 1.1153465509414673
Epoch: 270 Loss: 1.1151372194290161
Epoch: 271 Loss: 1.1149379014968872
Epoch: 272 Loss: 1.11472952365875

Epoch: 488 Loss: 1.0457550287246704
Epoch: 489 Loss: 1.0451984405517578
Epoch: 490 Loss: 1.0448740720748901
Epoch: 491 Loss: 1.0442719459533691
Epoch: 492 Loss: 1.0440768003463745
Epoch: 493 Loss: 1.0434057712554932
Epoch: 494 Loss: 1.043131709098816
Epoch: 495 Loss: 1.0428451299667358
Epoch: 496 Loss: 1.0421998500823975
Epoch: 497 Loss: 1.0418078899383545
Epoch: 498 Loss: 1.0413739681243896
Epoch: 499 Loss: 1.0407633781433105
Epoch: 500 Loss: 1.0405193567276
Epoch: 501 Loss: 1.0398519039154053
Epoch: 502 Loss: 1.039496898651123
Epoch: 503 Loss: 1.039133906364441
Epoch: 504 Loss: 1.0385816097259521
Epoch: 505 Loss: 1.0380946397781372
Epoch: 506 Loss: 1.03773832321167
Epoch: 507 Loss: 1.0377277135849
Epoch: 508 Loss: 1.0373311042785645
Epoch: 509 Loss: 1.0368870496749878
Epoch: 510 Loss: 1.0361356735229492
Epoch: 511 Loss: 1.035430908203125
Epoch: 512 Loss: 1.0351029634475708
Epoch: 513 Loss: 1.034774661064148
Epoch: 514 Loss: 1.0341377258300781
Epoch: 515 Loss: 1.0336881875991821
Epoch

Epoch: 718 Loss: 0.909240186214447
Epoch: 719 Loss: 0.9088436365127563
Epoch: 720 Loss: 0.9077904224395752
Epoch: 721 Loss: 0.9069192409515381
Epoch: 722 Loss: 0.9058316946029663
Epoch: 723 Loss: 0.905156135559082
Epoch: 724 Loss: 0.9049885869026184
Epoch: 725 Loss: 0.9035794138908386
Epoch: 726 Loss: 0.902678370475769
Epoch: 727 Loss: 0.901879608631134
Epoch: 728 Loss: 0.9017736911773682
Epoch: 729 Loss: 0.9002408981323242
Epoch: 730 Loss: 0.8996602296829224
Epoch: 731 Loss: 0.898417055606842
Epoch: 732 Loss: 0.8977149128913879
Epoch: 733 Loss: 0.8966993689537048
Epoch: 734 Loss: 0.8957420587539673
Epoch: 735 Loss: 0.8955303430557251
Epoch: 736 Loss: 0.8945257067680359
Epoch: 737 Loss: 0.8938748240470886
Epoch: 738 Loss: 0.8933764100074768
Epoch: 739 Loss: 0.8921363353729248
Epoch: 740 Loss: 0.8910987377166748
Epoch: 741 Loss: 0.8906483054161072
Epoch: 742 Loss: 0.8901739716529846
Epoch: 743 Loss: 0.8887763023376465
Epoch: 744 Loss: 0.8878887891769409
Epoch: 745 Loss: 0.88735097646713

Epoch: 950 Loss: 0.7094378471374512
Epoch: 951 Loss: 0.7076898813247681
Epoch: 952 Loss: 0.7072908878326416
Epoch: 953 Loss: 0.7060798406600952
Epoch: 954 Loss: 0.7067644000053406
Epoch: 955 Loss: 0.705863356590271
Epoch: 956 Loss: 0.7027121186256409
Epoch: 957 Loss: 0.7027803063392639
Epoch: 958 Loss: 0.7024510502815247
Epoch: 959 Loss: 0.701510488986969
Epoch: 960 Loss: 0.7002810835838318
Epoch: 961 Loss: 0.6985384225845337
Epoch: 962 Loss: 0.6972901821136475
Epoch: 963 Loss: 0.6979515552520752
Epoch: 964 Loss: 0.6962360143661499
Epoch: 965 Loss: 0.6942340731620789
Epoch: 966 Loss: 0.6949057579040527
Epoch: 967 Loss: 0.6932657957077026
Epoch: 968 Loss: 0.6932633519172668
Epoch: 969 Loss: 0.6910483837127686
Epoch: 970 Loss: 0.6904682517051697
Epoch: 971 Loss: 0.6905803084373474
Epoch: 972 Loss: 0.6890900135040283
Epoch: 973 Loss: 0.6874600648880005
Epoch: 974 Loss: 0.6875350475311279
Epoch: 975 Loss: 0.6847154498100281
Epoch: 976 Loss: 0.6851224303245544
Epoch: 977 Loss: 0.68296670913

...

Epoch: 9078 Loss: 0.006356887985020876
Epoch: 9079 Loss: 0.006357159931212664
Epoch: 9080 Loss: 0.0063554029911756516
Epoch: 9081 Loss: 0.006353707052767277
Epoch: 9082 Loss: 0.00635291263461113
Epoch: 9083 Loss: 0.006351832766085863
Epoch: 9084 Loss: 0.006350635085254908
Epoch: 9085 Loss: 0.006349607836455107
Epoch: 9086 Loss: 0.006349457893520594
Epoch: 9087 Loss: 0.006346843205392361
Epoch: 9088 Loss: 0.006346574984490871
Epoch: 9089 Loss: 0.006344950757920742
Epoch: 9090 Loss: 0.006344253197312355
Epoch: 9091 Loss: 0.0063424850814044476
Epoch: 9092 Loss: 0.006340965162962675
Epoch: 9093 Loss: 0.006341191940009594
Epoch: 9094 Loss: 0.006339003331959248
Epoch: 9095 Loss: 0.006338975392282009
Epoch: 9096 Loss: 0.006338008213788271
Epoch: 9097 Loss: 0.006336623802781105
Epoch: 9098 Loss: 0.006334955804049969
Epoch: 9099 Loss: 0.006333670113235712
Epoch: 9100 Loss: 0.006334078498184681
Epoch: 9101 Loss: 0.006331952754408121
Epoch: 9102 Loss: 0.00633123517036438
Epoch: 9103 Loss: 0.00632

Epoch: 9295 Loss: 0.006125593092292547
Epoch: 9296 Loss: 0.006123656406998634
Epoch: 9297 Loss: 0.006122896447777748
Epoch: 9298 Loss: 0.006122626829892397
Epoch: 9299 Loss: 0.0061214943416416645
Epoch: 9300 Loss: 0.006121013313531876
Epoch: 9301 Loss: 0.006119066849350929
Epoch: 9302 Loss: 0.006118922494351864
Epoch: 9303 Loss: 0.006116688251495361
Epoch: 9304 Loss: 0.006116311531513929
Epoch: 9305 Loss: 0.0061148423701524734
Epoch: 9306 Loss: 0.006114054936915636
Epoch: 9307 Loss: 0.006114463321864605
Epoch: 9308 Loss: 0.006112089846283197
Epoch: 9309 Loss: 0.006111303810030222
Epoch: 9310 Loss: 0.0061098262667655945
Epoch: 9311 Loss: 0.0061086248606443405
Epoch: 9312 Loss: 0.006108740344643593
Epoch: 9313 Loss: 0.006106373388320208
Epoch: 9314 Loss: 0.00610617408528924
Epoch: 9315 Loss: 0.0061042155139148235
Epoch: 9316 Loss: 0.006103119347244501
Epoch: 9317 Loss: 0.006103330757468939
Epoch: 9318 Loss: 0.006102113053202629
Epoch: 9319 Loss: 0.006102150306105614
Epoch: 9320 Loss: 0.0

Epoch: 9510 Loss: 0.005911143962293863
Epoch: 9511 Loss: 0.0059092105366289616
Epoch: 9512 Loss: 0.005908491089940071
Epoch: 9513 Loss: 0.005908147431910038
Epoch: 9514 Loss: 0.005906596779823303
Epoch: 9515 Loss: 0.0059050749987363815
Epoch: 9516 Loss: 0.00590494554489851
Epoch: 9517 Loss: 0.005902728997170925
Epoch: 9518 Loss: 0.0059036873281002045
Epoch: 9519 Loss: 0.005900898482650518
Epoch: 9520 Loss: 0.005901938304305077
Epoch: 9521 Loss: 0.005899510346353054
Epoch: 9522 Loss: 0.005899282172322273
Epoch: 9523 Loss: 0.005897668190300465
Epoch: 9524 Loss: 0.005896538961678743
Epoch: 9525 Loss: 0.005896789487451315
Epoch: 9526 Loss: 0.005894597619771957
Epoch: 9527 Loss: 0.0058950399979949
Epoch: 9528 Loss: 0.005892601795494556
Epoch: 9529 Loss: 0.005891821812838316
Epoch: 9530 Loss: 0.005891337525099516
Epoch: 9531 Loss: 0.005890403408557177
Epoch: 9532 Loss: 0.005888273473829031
Epoch: 9533 Loss: 0.005889259744435549
Epoch: 9534 Loss: 0.005886835046112537
Epoch: 9535 Loss: 0.00588

Epoch: 9726 Loss: 0.005707161966711283
Epoch: 9727 Loss: 0.005707773379981518
Epoch: 9728 Loss: 0.005706469528377056
Epoch: 9729 Loss: 0.0057061295956373215
Epoch: 9730 Loss: 0.005703909322619438
Epoch: 9731 Loss: 0.005702886264771223
Epoch: 9732 Loss: 0.0057031139731407166
Epoch: 9733 Loss: 0.005700865760445595
Epoch: 9734 Loss: 0.005700943060219288
Epoch: 9735 Loss: 0.00569938775151968
Epoch: 9736 Loss: 0.005698772147297859
Epoch: 9737 Loss: 0.005697845481336117
Epoch: 9738 Loss: 0.005697394721210003
Epoch: 9739 Loss: 0.005695374216884375
Epoch: 9740 Loss: 0.005695232655853033
Epoch: 9741 Loss: 0.0056941211223602295
Epoch: 9742 Loss: 0.005693354643881321
Epoch: 9743 Loss: 0.0056917411275208
Epoch: 9744 Loss: 0.005692485254257917
Epoch: 9745 Loss: 0.005690186750143766
Epoch: 9746 Loss: 0.005689856130629778
Epoch: 9747 Loss: 0.0056886072270572186
Epoch: 9748 Loss: 0.005687990225851536
Epoch: 9749 Loss: 0.005686576012521982
Epoch: 9750 Loss: 0.005686525721102953
Epoch: 9751 Loss: 0.0056

Epoch: 9940 Loss: 0.005520599894225597
Epoch: 9941 Loss: 0.005518798716366291
Epoch: 9942 Loss: 0.00551735982298851
Epoch: 9943 Loss: 0.00551648298278451
Epoch: 9944 Loss: 0.005515474826097488
Epoch: 9945 Loss: 0.005514466669410467
Epoch: 9946 Loss: 0.005514422431588173
Epoch: 9947 Loss: 0.005513876210898161
Epoch: 9948 Loss: 0.005512660834938288
Epoch: 9949 Loss: 0.005511559545993805
Epoch: 9950 Loss: 0.005510726943612099
Epoch: 9951 Loss: 0.005510149989277124
Epoch: 9952 Loss: 0.005508871283382177
Epoch: 9953 Loss: 0.005508180242031813
Epoch: 9954 Loss: 0.005506856366991997
Epoch: 9955 Loss: 0.005506709683686495
Epoch: 9956 Loss: 0.00550561398267746
Epoch: 9957 Loss: 0.005505741573870182
Epoch: 9958 Loss: 0.005503480788320303
Epoch: 9959 Loss: 0.005502481013536453
Epoch: 9960 Loss: 0.005502385552972555
Epoch: 9961 Loss: 0.005501691251993179
Epoch: 9962 Loss: 0.005500548519194126
Epoch: 9963 Loss: 0.005500024650245905
Epoch: 9964 Loss: 0.005498361773788929
Epoch: 9965 Loss: 0.00549833

最后我们用训练好的模型尝试在1到100这些数字上玩FizzBuzz游戏

In [8]:
# Output now
if torch.cuda.is_available():
    testX = torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(1, 101)]).cuda()
with torch.no_grad():
        testY = model(testX)
predictions = zip(range(1, 101), list(testY.max(1)[1].data.tolist()))

print([fizz_buzz_decode(i, x) for (i, x) in predictions])

['1', '2', 'fizz', '4', 'buzz', 'fizz', '7', '8', 'fizz', 'buzz', '11', 'fizz', '13', '14', 'fizzbuzz', '16', '17', 'fizz', '19', 'buzz', 'fizz', '22', '23', 'fizz', 'buzz', '26', 'fizz', '28', '29', 'fizzbuzz', '31', '32', '33', '34', 'buzz', '36', '37', '38', 'fizz', 'buzz', '41', 'fizz', '43', '44', 'fizzbuzz', '46', '47', 'fizz', '49', 'buzz', 'fizz', '52', 'fizz', 'fizz', 'buzz', '56', 'fizz', '58', '59', 'fizzbuzz', '61', '62', 'fizz', '64', 'buzz', 'fizz', '67', '68', '69', 'buzz', '71', 'fizz', '73', '74', 'fizzbuzz', '76', '77', 'fizz', '79', 'buzz', '81', '82', '83', 'buzz', 'buzz', '86', '87', '88', '89', 'fizzbuzz', '91', '92', '93', '94', 'buzz', 'fizz', '97', '98', 'fizz', 'buzz']


In [9]:
print(np.sum(testY.max(1)[1].cpu().numpy() == np.array([fizz_buzz_encode(i) for i in range(1,101)])))
testY.max(1)[1].cpu().numpy() == np.array([fizz_buzz_encode(i) for i in range(1,101)])

92


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True, False,  True,  True, False,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True])

## 小结

- binary_encode()是一个可以将十进制转化为任意位数二进制的函数
- 数组用法：[str(i), "fizz", "buzz", "fizzbuzz"][prediction],第一个[]为list，第二个[]为列表索引，相当于list[i]
- 分类问题一般使用交叉熵函数，回归问题一般使用均方误差函数作为loss function。交叉熵函数的输出为所有类别的概率分布。
- zip()函数可以将连个list打包成一个元组[Python3 zip() 函数](https://www.runoob.com/python3/python3-func-zip.html)